  # Analisis de datos en trabajadores activos

Este análisis es para ver como esta distribuido la población de trabajadores que cotizan para el issste y en que intervalo de edad esta 
concentrado y poner énfasis en el desarrollo de programas de atencion medica para este grupo de edad. 

In [ ]:
from scipy import stats
import pandas as pd
import seaborn as sns
import mysql.connector
import matplotlib.pyplot as plt
import datetime as dt

### Primera parte. Adquisicion del dataset

Procedemos a conectarnos a la base de datos:

In [ ]:
cnx = mysql.connector.connect(
    host="localhost",
    port=3306,
    user="root",
    password="******",
    database="fuentes_brot"
)

In [ ]:
cursor = cnx.cursor()

Y vemos que tablas contiene:

In [ ]:
pd.read_sql_query("show tables", cnx)

Seleccionamos la de "titulares_fb que es la que nos interesa y procedemos a ver el contenido:

In [ ]:
df = pd.read_sql_query("SELECT CAP, FECHA_NACIMIENTO, APELLIDO_PATERNO, APELLIDO_MATERNO, NOMBRE, GENERO, TIPO_DE_DIRECTO, RAMO1, REGIMEN, ESTATUS, BENEFICIARIOS_REG from titulares_fb", cnx)
df

Se genera dataframe llamado "titulares_fb.csv" y se cierra conexion:

In [ ]:
df.to_csv('c:/users/dell/Documents/data analysis/modulo4/proyecto final1/titulares_fb.csv')

In [ ]:
cursor.close()

## Segunda parte. Procesamiento de datos

Teniendo el dataset con el que se va a trabajar se inicia con el procesamiento. Lo primero que se hace es abrir el dataset:

In [ ]:
df_1 = pd.read_csv('c:/users/dell/Documents/data analysis/modulo4/proyecto final1/titulares_fb.csv')
df_1

primero se hace el reindexado:

In [ ]:
df_2 = df_1.set_index(pd.Series(range(1,424)))
df_2

Se ve que campos contiene el dataset:

In [ ]:
df_2.columns

se procede a realizar el filtrado para generar un dataframe con solo trabajadores activos:

In [ ]:
trab = df_2["TIPO_DE_DIRECTO"] == "TRABAJADOR"
trab

para sacar los trabajadores activos se necesita hacer filtrado tambien por ramo:

In [ ]:
trab1 = df_2["RAMO1"] != "CAJA DE PREV. PARA TRABS. A LISTA DE RAYA DEL GOB. DEL D.F."
trab1

In [ ]:
trab2 = df_2["RAMO1"] != "CONTINUACION VOLUNTARIA EN EL REGIMEN OBLIGATORIO"
trab2

In [ ]:
trab3 = df_2["RAMO1"] != "PENS. Y JUB. DE LA CAJA DE PREV. DE LA POLICIA PREVENTIVA DE"
trab3

In [ ]:
trab4 = df_2["ESTATUS"] == "VIGENTE"
trab4

In [ ]:
trab5 = df_2["CAP"] == "NO"
trab5

In [ ]:
tv = df_2[trab & trab1 & trab2 & trab3 & trab4 & trab5]
tv

In [ ]:
tv.columns

Se procede a eliminar datos sensibles y no esenciales para el estudio:

In [ ]:
tv_1 = tv.drop(columns=["Unnamed: 0", "CAP", "APELLIDO_PATERNO", "APELLIDO_MATERNO", "NOMBRE", "TIPO_DE_DIRECTO", "RAMO1", "ESTATUS"])
tv_1

Se necesita saber si hay datos nulos:

In [ ]:
tv_1.isna()

Y saber cuantos son:

In [ ]:
tv_1.isna().sum()

lo siguiente es ver que tipo de variables contiene cada campo:

In [ ]:
tv_1.dtypes

Como se observa el campo "FECHA_NACIMIENTO" hay que hacer cambio para que sea tipo datetime

In [ ]:
def nacimiento(valor):
    return pd.to_datetime(valor)

In [ ]:
tv_1["FECHA_NACIMIENTO"] = tv_1["FECHA_NACIMIENTO"].apply(nacimiento)
tv_1

Se verifica que se hizo la conversion:

In [ ]:
tv_1.dtypes

Se necesita generar dos campos más, uno que muestre la edad y otro que menciona en que intervalo de edad se encuentra:

In [ ]:
def longevidad (valor):
    hoy =dt.datetime.today()
    return  hoy.year - valor.year - ((hoy.month, hoy.day) < (valor.month, valor.day))

In [ ]:
tv_1["EDAD"] = tv_1["FECHA_NACIMIENTO"].apply(longevidad)
tv_1

In [ ]:
etiqueta = ['21 a 30','31 a 40','41 a 50','51 a 60','61 a 70','71 a 80','81 a 90','91 a 100','100 y mas']
tv_1['INTERVALO_EDAD'] = pd.cut(x = tv_1['EDAD'], bins = [20, 30, 40, 50, 60, 70, 80, 90 ,100, 110], labels = etiqueta)
tv_1

Checamos de nuevo los tipos de variable

In [ ]:
tv_1.dtypes

Se tiene el dataset que se necesita para hacer el análisis de datos

### Tercera parte. Análisis de datos

Teniendo ya el dataset que se necesita se procede a sacar primero la grafica para la cantidad de trabajadores por genero:

In [ ]:
tv_2 = tv_1
counts = tv_1['GENERO'].value_counts() 
counts

In [ ]:
sns.set_theme(style = 'whitegrid')
ax = sns.barplot(x=counts.index, y=counts)
ax.set_title('Poblacion de trabajadores activos por genero', fontsize=15, pad=5)
ax.set(ylabel='conteo');
ax.set(xlabel='genero');
ax.set_xticklabels(ax.get_xticklabels(), rotation=70);

En porcentaje sería:

In [ ]:
porcentaje = counts * 100 / counts.sum()
porcentaje

In [ ]:
sns.set_theme(style = 'whitegrid')
ax = sns.barplot(x=porcentaje.index, y=porcentaje)
ax.set_title('Poblacion de trabajadores activos por genero (%)', fontsize=15, pad=5)
ax.set(ylabel='%');
ax.set(xlabel='genero');
ax.set_xticklabels(ax.get_xticklabels(), rotation=70);

Ahora queremos saber en que intervalos de edad estan distribuidos los trajajadores

In [ ]:
counts_1 = tv_2['INTERVALO_EDAD'].value_counts()
counts_1

In [ ]:
ax = plt.subplots(figsize=(10, 6))
ax = sns.barplot(x= counts_1.index, y= counts_1)
ax.set_title('Distribución de trabajadores activos por intervalo de edades', fontsize=15, pad=5)
ax.set(xlabel='rango de edades');
ax.set(ylabel='conteo');

Esta misma dsitribución se muestra ahora por genero:

In [ ]:
tv_2['count'] = 1
tv_2

In [ ]:
counts_2 = tv_2.groupby(['INTERVALO_EDAD', 'GENERO'])['count'].sum()
counts_2

In [ ]:
counts_2 = counts_2.unstack(1)
counts_2

In [ ]:
plt.style.use('seaborn')

fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot()

plt1 = ax.bar(counts_2.index, counts_2['HOMBRE'], label='HOMBRE')
plt2 = ax.bar(counts_2.index, counts_2['MUJER'], bottom= counts_2['HOMBRE'])

ax.set_ylabel('conteo')
ax.set_title('Número de beneficiarios segun intervalos de edad y genero', fontsize=15, pad=5);
plt.legend((plt1[0], plt2[0]), ('Hombre', 'Mujer'));
ax.set_ylim(0, 70);

Le sacamos l media, la media truncada y la mediana para saber en que edad anda la poblacion de trabajadores activos

In [ ]:
mt = stats.trim_mean(tv_2["EDAD"], 0.1)
print(f'La media truncada es: {mt:,.2f} años ')

In [ ]:
media = tv_2["EDAD"].mean()
print(f'La media es: {media:,.2f} años ')

In [ ]:
media = tv_2["EDAD"].median()

print(f'La mediana es: {media:,.2f} años ')

In [ ]:
print(f'La desviación estandar es: { tv_2["EDAD"].std()}')

In [ ]:
tv_2["EDAD"].mode()

In [ ]:
print(f'Valor mínimo: {tv_2["EDAD"].min()}')
print(f'Percentil 0: {tv_2["EDAD"].quantile(0)}')
print(f'Percentil 10: {tv_2["EDAD"].quantile(0.1)}')
print(f'Percentil 25: {tv_2["EDAD"].quantile(0.25)}')
print(f'Percentil 50: {tv_2["EDAD"].median()}')
print(f'Mediana: {tv_2["EDAD"].quantile(0.5)}')
print(f'Percentil 75: {tv_2["EDAD"].quantile(0.75)}')
print(f'Percentil 90: {tv_2["EDAD"].quantile(0.9)}')
print(f'Percentil 100: {tv_2["EDAD"].quantile(1)}')
print(f'Valor máximo: {tv_2["EDAD"].max()}')

In [ ]:
print(f'El rango intercuartilico se encuentra en: {tv_2["EDAD"].quantile(0.75) - tv_2["EDAD"].quantile(0.25)}')

Se realiza un boxplot para ver como estan distribuidos los cuartiles:

In [ ]:
sns.set(style="whitegrid")
sns.boxplot(x=tv_2["EDAD"])
plt.axvline(tv_2["EDAD"].mean(), c='r');

Como se ve en el boxplot la mediana y media estan casi a la par y no hay valores atipicos el el dataset. Por último queremos ver cual es la correlación entre la edad de los trabajdores y el número de beneficiarios registrados por estos.

In [ ]:
sns.set(style="whitegrid")
plt.scatter(x=tv_2['EDAD'], y= tv_2['BENEFICIARIOS_REG'], c='m');

In [ ]:
relacion = tv_2['EDAD'].corr(tv_2['BENEFICIARIOS_REG'] )
print(f'Correlación entre las edad y beneficiaros dados de alta: { relacion }' )

## Conclusiones:

El dataset que se utilizo fue construido manualmente y hasta el momento no ha sido terminado, pero permitió dar un pequeño bosquejo sobre 
el intervalo en edades en la que esta actualmente la clinica en donde se labora, y en un futuro terminado poder mostrar datos más realistas.
